<a href="https://colab.research.google.com/github/marstival/tesourodireto/blob/main/src/TesouroPrecoTaxa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
'''
#download to ../data/ 
import io

from urllib.request import urlretrieve
url = "https://www.tesourotransparente.gov.br/ckan/dataset/df56aa42-484a-4a59-8184-7676580c81e3/resource/796d2059-14e9-44e3-80c9-2d9e30b405c1/download/PrecoTaxaTesouroDireto.csv"
path, headers = urlretrieve(url, "../data/PrecoTaxaTesouroDireto.csv")

for name, value in headers.items():
  print(name, value)

'''

In [21]:
'''
#load csv from ../data
import pandas as pd
df = pd.read_csv('../data/PrecoTaxaTesouroDireto.csv', encoding='latin-1', sep=';', decimal=',')
'''

In [ ]:
# prompt: download and load the following csv file into a pandas dataframe: 
# https://www.tesourotransparente.gov.br/ckan/dataset/df56aa42-484a-4a59-8184-7676580c81e3/resource/796d2059-14e9-44e3-80c9-2d9e30b405c1/download/PrecoTaxaTesouroDireto.csv

from google.colab import files
import pandas as pd

uploaded = files.upload()

df = pd.read_csv('PrecoTaxaTesouroDireto.csv', encoding='latin-1', sep=';', decimal=',')

In [22]:
df['Data Vencimento'] = pd.to_datetime(df['Data Vencimento'], format='%d/%m/%Y')
df['Data Base'] = pd.to_datetime(df['Data Base'], format='%d/%m/%Y')

df['PU Venda Manha'] = pd.to_numeric(df['PU Venda Manha'],  errors='coerce')
df['Taxa Compra Manha'] = pd.to_numeric(df['Taxa Compra Manha'], errors='coerce')

# Print the dataframe
#print(df)


In [23]:
# prompt: from the dataframe df, extract distinct values for the combinations of  'Tipo Titulo' and 'Data Vencimento' into a dict to keep the hierarchy

def extract_unique_combinations(df):
  """
  Extracts distinct values for the combinations of two columns into a dictionary.

  Args:
      df: A Pandas DataFrame.

  Returns:
      A dictionary where keys are tuples of unique values from the first column
      and values are lists of unique values from the second column.
  """

  df = df.sort_values(by=['Tipo Titulo', 'Data Vencimento'])
  unique_combinations = {}
  unique_combinations["Selecione..."] = []
  for index, row in df.iterrows():
    key = (row["Tipo Titulo"])
    if key not in unique_combinations:
      unique_combinations[key] = []

    dt_vencimento = row["Data Vencimento"].strftime('%Y')
    if dt_vencimento not in unique_combinations[key]:
      unique_combinations[key].append( dt_vencimento )
  return unique_combinations

unique_combinations = extract_unique_combinations(df)
#print(unique_combinations)


In [24]:
# prompt: use widgets to allow the user to select the options from the dictionary unique_combinations above. the first input allow the user to select the key, and the second input to select one of the values of that key

from ipywidgets import Dropdown, HBox, VBox


# Create a dropdown for the first level of keys
key_dropdown = Dropdown(
    options=list(unique_combinations.keys() ),
    description='Tipo Titulo:',
)

# Create a dropdown for the second level of values
value_dropdown = Dropdown(
    options=[],
    description='Data Vencimento:',
)

# Update the second dropdown based on the selected key
def update_value_dropdown(change):
    selected_key = change.new
    value_dropdown.options = unique_combinations[selected_key]

# Connect the two dropdowns
key_dropdown.observe(update_value_dropdown, names='value')

# Display the dropdowns
VBox([key_dropdown, value_dropdown])


In [25]:
# prompt: obtain the selected options for the two dropdowns above

tipo_titulo = key_dropdown.value
vencimento_titulo = value_dropdown.value

print(f"Selected key: {tipo_titulo}")
print(f"Selected value: {vencimento_titulo}")


Selected key: Tesouro Educa+
Selected value: 2040


In [26]:
# prompt: filter the dataframe df for 'Tipo Titulo'  com valor "Tesouro IPCA+' e Data Vencimento no ano de 2029

filtered_df = df[
    (df['Tipo Titulo'] == tipo_titulo) & (df['Data Vencimento'].dt.year == int(vencimento_titulo))
]

#print(filtered_df)


In [27]:
# prompt: order the dataframe by Data Base and plot a chart using plotly lib showing the values of Taxa Venda Manha and PU Compra Manha in different Y axis and  Data Base as the x axis

import plotly.graph_objects as go
from plotly.subplots import make_subplots # Import the make_subplots function


# Sort the dataframe by Data Base
filtered_df = filtered_df.sort_values(by=['Data Base'])

# Create a figure with two subplots
title_tx = "Taxa Compra {} {}".format(tipo_titulo, vencimento_titulo)
title_pu = "PU Compra {} {}".format(tipo_titulo, vencimento_titulo)
print(title_tx)

fig = make_subplots(rows=1, cols=2, subplot_titles=(title_tx, title_pu))


# Add traces for Taxa Venda Manha and PU Compra Manha
fig.add_trace(go.Scatter(x=filtered_df['Data Base'], y=filtered_df['Taxa Compra Manha'], name="Taxa Compra Manha"), row=1, col=1)
fig.add_trace(go.Scatter(x=filtered_df['Data Base'], y=filtered_df['PU Compra Manha'], name="PU Compra Manha", yaxis="y2"), row=1, col=2)

# Configure y-axis titles and show the figure
fig.update_yaxes(title_text="Taxa Compra Manha", row=1, col=1)
fig.update_yaxes(title_text="PU Compra Manha", row=1, col=2)
fig.show()


Taxa Compra Tesouro Educa+ 2040


In [28]:
# prompt: save fig to hml file

fig.write_html('fig1.html', auto_open=True)
